In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import requests
import json

In [8]:
plotly.tools.set_credentials_file(username='wangdian', api_key='DKvDkKjnNZWzlT6zhaCq')
mapbox_access_token = 'pk.eyJ1Ijoid2FuZ2RpYW5kaWFuIiwiYSI6ImNqb3JyeWV4czAyeWwzcW85enRyZmp5dm8ifQ.UIVacLewjXi3XRL2XvlsgA'

In [9]:
# map plot location
path = 'data/MCI_2014_to_2017.csv'

df = pd.read_csv(path)
df.head()

,X,Y,Index_,event_unique_id,occurrencedate,reporteddate,premisetype,ucr_code,ucr_ext,offence,...,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,Neighbourhood,Lat,Long,FID
0,-79.520401,43.768829,14601,GO-20142775022,2014-08-25T04:00:00.000Z,2014-08-25T04:00:00.000Z,Outside,1430,100,Assault,...,237.0,Monday,18,Assault,D31,24,Black Creek (24),43.768829,-79.520401,14001
1,-79.580856,43.642574,14602,GO-20142870874,2014-08-25T04:00:00.000Z,2014-09-08T04:00:00.000Z,House,2120,220,B&E W'Intent,...,237.0,Monday,9,Break and Enter,D22,11,Eringate-Centennial-West Deane (11),43.642574,-79.580856,14002
2,-79.260445,43.762909,14603,GO-20142802386,2014-08-25T04:00:00.000Z,2014-08-29T04:00:00.000Z,House,1430,100,Assault,...,237.0,Monday,11,Assault,D41,127,Bendale (127),43.762909,-79.260445,14003
3,-79.367546,43.663208,14604,GO-20142777955,2014-08-25T04:00:00.000Z,2014-08-26T04:00:00.000Z,Commercial,2120,200,B&E,...,237.0,Monday,17,Break and Enter,D51,71,Cabbagetown-South St.James Town (71),43.663208,-79.367546,14004
4,-79.231758,43.776440,14605,GO-20142778699,2014-08-25T04:00:00.000Z,2014-08-26T04:00:00.000Z,Other,2120,200,B&E,...,237.0,Monday,16,Break and Enter,D43,137,Woburn (137),43.776440,-79.231758,14005


In [10]:
df = df.sort_values(by=['Hood_ID'])
x = df.loc[df['Hood_ID'] == 1]['X'].mean()

In [11]:
map_id_dict = {
    'hood_id': df['Hood_ID'].unique(),
    'lon': df.groupby('Hood_ID')['X'].mean(),
    'lan': df.groupby('Hood_ID')['Y'].mean(),
    'neighbourhood': df.groupby('Hood_ID')['Neighbourhood'].unique()
}

map_id_df = pd.DataFrame(map_id_dict, index=map_id_dict['hood_id'])

In [12]:
map_id_df.head()

,hood_id,lon,lan,neighbourhood
1,1,-79.598015,43.719981,[West Humber-Clairville (1)]
2,2,-79.588351,43.745059,[Mount Olive-Silverstone-Jamestown (2)]
3,3,-79.568820,43.738668,[Thistletown-Beaumond Heights (3)]
4,4,-79.566224,43.720970,[Rexdale-Kipling (4)]
5,5,-79.551941,43.720902,[Elms-Old Rexdale (5)]


In [13]:
# total crime
path = 'data/crime_totals_per_hood.csv'

crime_total_df = pd.read_csv(path, header=None)
crime_total_df.columns = ['hood_id', 'total_crime']
crime_total_df.head()

,hood_id,total_crime
0,1,3264
1,2,1806
2,3,475
3,4,509
4,5,444


In [14]:
crime_total_df = crime_total_df.merge(map_id_df, on='hood_id', how='outer' )
crime_total_df.head()

,hood_id,total_crime,lon,lan,neighbourhood
0,1,3264,-79.598015,43.719981,[West Humber-Clairville (1)]
1,2,1806,-79.588351,43.745059,[Mount Olive-Silverstone-Jamestown (2)]
2,3,475,-79.568820,43.738668,[Thistletown-Beaumond Heights (3)]
3,4,509,-79.566224,43.720970,[Rexdale-Kipling (4)]
4,5,444,-79.551941,43.720902,[Elms-Old Rexdale (5)]


In [15]:
crime_total_map_plot = []
for index, row in crime_total_df.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['lan'])],
            lon=[str(row['lon'])],
            mode='markers',
            marker=dict(
                size=0.02 * row['total_crime'],
                opacity= 0.7,
                color = 'rgb(17, 157, 255)'
            ),
            
            name=row['neighbourhood'][0],
            text=row['neighbourhood'][0] # pending disctrict name
        )
    
    crime_total_map_plot.append(data)
    
layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=43.7,
            lon=-79.5
        ),
        pitch=0,
        zoom=10
    ),
)

fig = dict(data=crime_total_map_plot, layout=layout)

py.iplot(fig, filename='Toronto Crime')

In [16]:
#
crime_path_3 = 'data/out.csv'

crime_accnum_df = pd.read_csv(crime_path_3)

In [17]:
crime_accnum_df = crime_accnum_df.merge(map_id_df, on='hood_id', how='outer')
crime_accnum_df.head()

,hood_id,accnum,lon,lan,neighbourhood
0,1.0,427,-79.598015,43.719981,[West Humber-Clairville (1)]
1,2.0,106,-79.588351,43.745059,[Mount Olive-Silverstone-Jamestown (2)]
2,3.0,82,-79.568820,43.738668,[Thistletown-Beaumond Heights (3)]
3,4.0,74,-79.566224,43.720970,[Rexdale-Kipling (4)]
4,5.0,19,-79.551941,43.720902,[Elms-Old Rexdale (5)]


In [18]:
# create data for crime_accnum_map_plot
crime_accnum_map_plot = []
for index, row in crime_accnum_df.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['lan'])],
            lon=[str(row['lon'])],
            mode='markers',
            marker=dict(
                size=0.2 * row['accnum'],
                opacity= 0.7,
                color = 'coral'
            ),
            
            name=row['neighbourhood'][0],
            text=row['neighbourhood'][0] # pending disctrict name
        )
    
    crime_accnum_map_plot.append(data)
    

fig = dict(data=crime_accnum_map_plot, layout=layout)

py.iplot(fig, filename='Toronto Accnum')

In [19]:
df['MCI'].value_counts()

Assault            71440
Break and Enter    27353
Robbery            14591
Auto Theft         13480
Theft Over          4209
Name: MCI, dtype: int64

In [20]:
mci_color_map = {
    'Assault': 'coral',
    'Auto Theft': 'gold',
    'Break and Enter': 'blue',
    'Theft Over': 'purple',
    'Robbery': 'green'
}

In [21]:
theft_over_df = df.loc[df['MCI'] == 'Theft Over']

In [22]:
theft_over_2017 = theft_over_df.loc[theft_over_df['reportedyear'] == 2017]
theft_over_2017_final = theft_over_2017[['X', 'Y', 'premisetype', 'Neighbourhood']]
theft_over_2017_commercial = theft_over_2017_final.loc[theft_over_2017_final['premisetype'] == 'Commercial']
theft_over_2017_commercial.head()

,X,Y,premisetype,Neighbourhood
89157,-79.570541,43.697189,Commercial,West Humber-Clairville (1)
89311,-79.614281,43.745495,Commercial,West Humber-Clairville (1)
88048,-79.597366,43.690762,Commercial,West Humber-Clairville (1)
87060,-79.570580,43.691063,Commercial,West Humber-Clairville (1)
93960,-79.598656,43.693562,Commercial,West Humber-Clairville (1)


In [23]:
theft_over_2017_final['premisetype'].value_counts()

Commercial    395
Outside       283
House         217
Apartment     185
Other          88
Name: premisetype, dtype: int64

In [24]:
theft_over_2017_final.index = theft_over_2017_final['premisetype']
theft_over_2017_indoor = theft_over_2017_final.loc[['House', 'Apartment']]
theft_over_2017_indoor.head()

,X,Y,premisetype,Neighbourhood
premisetype,,,,
House,-79.620079,43.722992,House,West Humber-Clairville (1)
House,-79.618431,43.719708,House,West Humber-Clairville (1)
House,-79.618431,43.719708,House,West Humber-Clairville (1)
House,-79.553352,43.740631,House,Thistletown-Beaumond Heights (3)
House,-79.572472,43.728905,House,Rexdale-Kipling (4)


In [25]:
theft_2017_commercial_data = []

for index, row in theft_over_2017_commercial.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.7,
                color = 'coral'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_commercial_data.append(data)

In [26]:
fig = dict(data=theft_2017_commercial_data, layout=layout)

py.iplot(fig, filename='Toronto Theft Over Plot 2017 Commercial')

In [27]:
# indoor theft 2017
theft_2017_indoor_data = []

for index, row in theft_over_2017_indoor.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.7,
                color = 'purple'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_indoor_data.append(data)

In [28]:
fig = dict(data=theft_2017_indoor_data, layout=layout)

py.iplot(fig, filename='Toronto Theft Over Plot 2017 Indoor')

In [29]:
theft_over_2017_outdoor = theft_over_2017_final.loc[['Outside']]

In [30]:
# outside theft 2017
theft_2017_outdoor_data = []

for index, row in theft_over_2017_outdoor.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.5,
                color = 'black'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_outdoor_data.append(data)

In [31]:
fig = dict(data=theft_2017_outdoor_data, layout=layout)

py.iplot(fig, filename='Toronto Theft Over Plot 2017 Outside')

In [32]:
theft_over_2017_other = theft_over_2017_final.loc[['Other']]

In [33]:
# other theft 2017
theft_2017_other_data = []

for index, row in theft_over_2017_other.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.6,
                color = 'blue'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_other_data.append(data)

In [34]:
fig = dict(data=theft_2017_other_data, layout=layout)
py.iplot(fig, filename='Toronto Theft Over Plot 2017 other')

In [36]:
theft_over_2017_house = theft_over_2017_final.loc[['House']]

In [37]:
# house theft 2017
theft_2017_house_data = []

for index, row in theft_over_2017_house.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.6,
                color = 'red'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_house_data.append(data)

In [39]:
fig = dict(data=theft_2017_house_data, layout=layout)
py.iplot(fig, filename='Toronto Theft Over Plot 2017 house')

In [45]:
theft_over_2017_apartment = theft_over_2017_final.loc[['Apartment']]

In [46]:
# apartment theft 2017
theft_2017_apartment_data = []

for index, row in theft_over_2017_apartment.iterrows():
    
    data = go.Scattermapbox(
            lat=[str(row['Y'])],
            lon=[str(row['X'])],
            mode='markers',
            marker=dict(
                size=8,
                opacity= 0.6,
                color = 'green'
            ),
            
            name=row['Neighbourhood'],
            text=row['Neighbourhood'] # pending disctrict name
        )
    
    theft_2017_apartment_data.append(data)

In [47]:
fig = dict(data=theft_2017_apartment_data, layout=layout)
py.iplot(fig, filename='Toronto Theft Over Plot 2017 apartment')